<a href="https://colab.research.google.com/github/neuroidss/EEG-GAN-audio-video/blob/main/bdf_coh_3d_viz.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%matplotlib inline

import matplotlib.pyplot as plt
import matplotlib
matplotlib.use('agg')

#for k in range(100):
#  fig, ax = plt.subplots(figsize=(37.33, 21))
#  fig.savefig(f'figure{k}.png')
#  plt.close(fig)

In [2]:
ch_names_sg2 = ['FP1','AF3','F7','F3','FC1','FC5','T7','C3','CP1','CP5','P7','P3','Pz','PO3','O1','Oz','O2','PO4','P4','P8','CP6','CP2','C4','T8','FC6','FC2','F4','F8','AF4','FP2','Fz','Cz']
ch_locations_sg2=[0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31]
bands = [[8.,12.]]
methods = ['coh']
duration=5*1/8
overlap=0
fps_sg2=1
#if generate_wavegan:
fps_wg=1#hz/(32768*2)
fps_sg2=fps_wg*4
#fps_sg2=fps_wg
fps_hm=fps_wg

if 2*1/fps_wg>duration:
  duration=2*1/fps_wg
  overlap=0

#if generate&gen_wavegan:
#  dim_wg = 100
#if generate&gen_stylegan2:
#  dim_sg2 = 512
#if generate&gen_sg2_shawwn:
#  dim_sg2 = 1024

#stepSize = 1/pow(2,24)
#vref = 2.50 #2.5V voltage ref +/- 250nV
#gain = 8

#vscale = (vref/gain)*stepSize #volts per step.
#uVperStep = 1000000 * ((vref/gain)*stepSize) #uV per step.
#scalar = 1/(1000000 / ((vref/gain)*stepSize)) #steps per uV.

In [3]:
def download_file_from_google_drive(file_id,dest_path):
  import os.path
  if not os.path.isfile(dest_path):  
    !mkdir -p $(dirname {dest_path})
    !wget --save-cookies cookies.txt 'https://docs.google.com/uc?export=download&id='{file_id} -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1/p' > confirm.txt
    !wget --load-cookies cookies.txt -O {dest_path} 'https://docs.google.com/uc?export=download&id='{file_id}'&confirm='$(<confirm.txt)

In [4]:
!mkdir /content/eeg

mkdir: cannot create directory ‘/content/eeg’: File exists


In [5]:
!pip install --upgrade gdown
#!pip install googledrivedownloader
#from google_drive_downloader import GoogleDriveDownloader as gdd
files_path=[]
#if generate&gen_drums:
files_path = [['1Nfzi6yT83SBZxtgIYVtYp7C7g_Sq9OdS', '/content/eeg/record-[2019.11.13-22.23.59].csv'],
              ['1LtMfr9GduR3semMVgh_6JoUSKbCG8XbH', '/content/eeg/record-[2020.06.28-14.26.09].csv']]
for i in range(len(files_path)):
#  gdd.download_file_from_google_drive(file_id=files_path[i][0], dest_path=files_path[i][1])
  download_file_from_google_drive(file_id=files_path[i][0], dest_path=files_path[i][1])

files_path = [['1nIiilGVq8XXU7bb1UC5GDtmGbovuil9C', '/content/eeg/5min_experienced_meditator_unfiltered_signals.bdf', '5min_experienced_meditator_unfiltered_signals']]

for i in range(len(files_path)):
#  gdd.download_file_from_google_drive(file_id=files_path[i][0], dest_path=files_path[i][1])
  download_file_from_google_drive(file_id=files_path[i][0], dest_path=files_path[i][1])

files_path = [['1BphDQFJZ0aIMiZbkQ55_MzNxOrdq0Jcb', '/content/eeg/01-07-2022_15-34.bdf', '01-07-2022_15-34'],
              ['1HlDuAO8n_hmH1PWlT46T1Waqyy5yz1LV', '/content/eeg/01-07-2022_18-32.bdf', '01-07-2022_18-32'],
              ['1u_8ANTzYB9jP7wCnErivF1ClM1rXixSg', '/content/eeg/01-07-2022_16-52.bdf', '01-07-2022_16-52'],
              ['1_cgoBcHAf3VRIorYafC-vHLAGpv8Papz', '/content/eeg/01-07-2022_17-39.bdf', '01-07-2022_17-39'],
              ['1JMLtO_sQtZtdQHQ2RCZ0RbRj3cvk0G_2', '/content/eeg/01-07-2022_14-05.bdf', '01-07-2022_14-05'],
              ['1ag-1yK-HSxNRh_5wnP9h-kOkVyX3YqSs', '/content/eeg/01-07-2022_18-32_signals.csv']]

for i in range(len(files_path)):
#  gdd.download_file_from_google_drive(file_id=files_path[i][0], dest_path=files_path[i][1])
  download_file_from_google_drive(file_id=files_path[i][0], dest_path=files_path[i][1])

files_path = [['16j4PwPoudiCVB_HF9m0fCYIt3uUZR3kA', '/content/eeg/20211130-162023-sync1.bdf', '20211130-162023-sync1'],
              ['16kSyskI7qOZRitvNIzd4iy6irqhgLwsI', '/content/eeg/20211130-162023-sync2.bdf', '20211130-162023-sync2'],
              ['16k88jpP7uZKKDT-WhZKcvGsnPclDA2TF', '/content/eeg/20211130-162023-sync3.bdf', '20211130-162023-sync3'],
              ['16licSohQPcbLnJmMD88In5PbiiLjOB_N', '/content/eeg/20211130-162023-sync4.bdf', '20211130-162023-sync4'],
              ['16nHtHn1BBkzb9oTl4EhJmPHPEB4Q8CVl', '/content/eeg/20211130-221138-sync1.bdf', '20211130-221138-sync1'],
              ['16m5Kp0WEdz3GixvQ7-jPVLdQUhXmIbA6', '/content/eeg/20211130-221138-sync2.bdf', '20211130-221138-sync2'],
              ['16mJVZV1XT-RULgcSKUSROOQnQr6CHYCK', '/content/eeg/20211130-221138-sync3.bdf', '20211130-221138-sync3'],
              ['16nh5R44VIRDFbYmFk7sDQeMXEAA1veY4', '/content/eeg/20211130-221138-sync4.bdf', '20211130-221138-sync4'],
              ['16oygYh5ZerFWmNHNMaFqZxUNbC0yj-oW', '/content/eeg/20211130-231131-sync1.bdf', '20211130-231131-sync1'],
              ['16q-Rl-L6NcTxdKco347Q63eK8Z3e5h-F', '/content/eeg/20211130-231131-sync2.bdf', '20211130-231131-sync2'],
              ['16o9MNod2FHz3tplGv14a8vxNR8EVGfxx', '/content/eeg/20211130-231131-sync3.bdf', '20211130-231131-sync3'],
              ['16oszc_L44xtZNOgBuEJtgSjwl1Pob825', '/content/eeg/20211130-231131-sync4.bdf', '20211130-231131-sync4']]

for i in range(len(files_path)):
#  gdd.download_file_from_google_drive(file_id=files_path[i][0], dest_path=files_path[i][1])
  download_file_from_google_drive(file_id=files_path[i][0], dest_path=files_path[i][1])


You should consider upgrading via the '/home/dmitryneuro/colab/env/bin/python3.7 -m pip install --upgrade pip' command.


In [6]:
!pip install mne==0.23.3

import mne
from mne import io

You should consider upgrading via the '/home/dmitryneuro/colab/env/bin/python3.7 -m pip install --upgrade pip' command.


In [7]:
    ch_names = ['Fz','Fp1','F7','F3','T3','C3','T5','P3','O1','Pz','O2','P4','T6','C4','T4','F4','F8','Fp2']
#    ch_names = ['Fp1','F7','F3','T3','C3','T5','P3','O1','Pz','O2','P4','T6','C4','T4','F4','F8','Fp2','Fz']
#    ch_names = ['F4','F8','Fp2','Fz','Fp1','F7','F3','T3','C3','T5','P3','O1','Pz','O2','P4','T6','C4','T4']
#    ch_names = ['Fp1','F7','F3','T3','C3','T5','P3','Pz','O1','O2','P4','T6','C4','T4','F4','F8','Fp2','Fz']
#    ch_names = ['FP1','F7','F3','T3','C3','T5','P3','PZ','O1','O2','P4','T6','C4','T4','F4','F8','FP2','FZ']
#    ch_names = ['O2','T6','T4','F8','FP2','F4','C4','P4','PZ','FZ','FP1','F3','C3','P3','O1','T5','T3','F7']

    sfreq = 512 
    ch_types=['eeg']*len(ch_names)
    info = mne.create_info(ch_names = ch_names, sfreq = sfreq, ch_types=ch_types)
    misc_ch_names = ['','']
    





In [8]:
    input_fname=files_path[6][1]
    input_fname_name=files_path[6][2]
#    raw = io.read_raw_bdf(input_fname, eog=None, misc=None, stim_channel='auto', exclude=(), preload=False, verbose=None,include)
    raw = io.read_raw_bdf(input_fname, eog=None, misc=None, stim_channel='auto', 
                          exclude=(), preload=False, verbose=True)


Extracting EDF parameters from /content/eeg/20211130-221138-sync3.bdf...
BDF file detected
Setting channel info structure...
Creating raw.info structure...


In [9]:
raw.info

<Info | 7 non-empty values
 bads: []
 ch_names: counter, timestamp, O2, T6, T4, F8, Fp2, F4, C4, P4, button, Pz, ...
 chs: 22 EEG
 custom_ref_applied: False
 highpass: 0.0 Hz
 lowpass: 256.0 Hz
 meas_date: 2021-11-30 22:11:57 UTC
 nchan: 22
 projs: []
 sfreq: 512.0 Hz
>

In [10]:
  !pip install Pillow
  import PIL.Image 
  !pip install tqdm
  from tqdm import tqdm
  !pip install imageio==2.4.1
  !pip install imageio-ffmpeg==0.4.3 pyspng==0.1.0


You should consider upgrading via the '/home/dmitryneuro/colab/env/bin/python3.7 -m pip install --upgrade pip' command.
You should consider upgrading via the '/home/dmitryneuro/colab/env/bin/python3.7 -m pip install --upgrade pip' command.
You should consider upgrading via the '/home/dmitryneuro/colab/env/bin/python3.7 -m pip install --upgrade pip' command.
You should consider upgrading via the '/home/dmitryneuro/colab/env/bin/python3.7 -m pip install --upgrade pip' command.


In [11]:
!mkdir '/content/out'


mkdir: cannot create directory ‘/content/out’: File exists


In [12]:
!pip install mne==0.23.3
!pip install pandas
!pip install matplotlib

import mne
from mne import io
from mne.datasets import sample
from mne.minimum_norm import read_inverse_operator, compute_source_psd

from mne.connectivity import spectral_connectivity, seed_target_indices

import pandas as pd
import numpy as np


You should consider upgrading via the '/home/dmitryneuro/colab/env/bin/python3.7 -m pip install --upgrade pip' command.
You should consider upgrading via the '/home/dmitryneuro/colab/env/bin/python3.7 -m pip install --upgrade pip' command.
You should consider upgrading via the '/home/dmitryneuro/colab/env/bin/python3.7 -m pip install --upgrade pip' command.


In [13]:
if False:
    key = 0
    idx = 0

    data_path = '/content/eeg'
    #raw_fname = data_path + '/record-[2019.11.13-22.23.59].gdf'
    #raw = mne.io.read_raw_gdf(raw_fname, preload=True)

    path = data_path

    ch_names = ['FP1','AF3','F7','F3','FC1','FC5','T7','C3','CP1','CP5','P7','P3','Pz','PO3','O1','Oz','O2','PO4','P4','P8','CP6','CP2','C4','T8','FC6','FC2','F4','F8','AF4','FP2','Fz','Cz']
    data = pd.read_csv(path + '/record-[2019.11.13-22.23.59].csv', skiprows=0, usecols=ch_names, header=0, delimiter=';') 
    
    #ch_names = ['FP1','F3','P3','O1','O2','P4','F4','FP2']
    #data = pd.read_csv(path + '/record-[2020.06.28-14.26.09].csv', skiprows=0, usecols=ch_names, header=0, delimiter=';') 
    
    #data = pd.read_csv(path + '/record-[2020.06.28-00.36.11].csv', skiprows=0, usecols=ch_names, header=0, delimiter=';') 
    #data = pd.read_csv(path + '/record-[2020.06.29-19.49.23].csv', skiprows=0, usecols=ch_names, header=0, delimiter=';') 
    
    #print(data)
    data_transpose=np.transpose(data)

    sfreq = 512 
    ch_types=['eeg']*len(ch_names)
    info = mne.create_info(ch_names = ch_names, sfreq = sfreq, ch_types=ch_types)
    #info = mne.create_info(sfreq = sfreq)
    raw = mne.io.RawArray(data_transpose, info)
    #raw.plot()
    
    input_fname=files_path[0][1]
    mne.io.read_raw_bdf(input_fname, eog=None, misc=None, stim_channel='auto', exclude=(), infer_types=False, include=None, preload=False, verbose=None)

    # Setup for reading the raw data
    #raw = io.read_raw_fif(raw_fname, verbose=False)
    #events = mne.find_events(raw, stim_channel='STI 014')
    #inverse_operator = read_inverse_operator(fname_inv)
    #raw.info['bads'] = ['MEG 2443', 'EEG 053']

    # picks MEG gradiometers
    #picks = mne.pick_types(raw.info, meg=False, eeg=True, eog=False, stim=False)
    picks = ch_names

In [14]:
raw.ch_names


['counter',
 'timestamp',
 'O2',
 'T6',
 'T4',
 'F8',
 'Fp2',
 'F4',
 'C4',
 'P4',
 'button',
 'Pz',
 'Fz',
 'sync',
 'Fp1',
 'F3',
 'C3',
 'P3',
 'O1',
 'T5',
 'T3',
 'F7']

In [15]:
print(raw)
raw.pick(ch_names)

<RawEDF | 20211130-221138-sync3.bdf, 22 x 882176 (1723.0 s), ~26 kB, data not loaded>


<RawEDF | 20211130-221138-sync3.bdf, 18 x 882176 (1723.0 s), ~22 kB, data not loaded>

In [16]:
#!pip install imageio==2.9
!pip install imageio==2.4.1
!pip install imageio-ffmpeg==0.4.3 pyspng==0.1.0

You should consider upgrading via the '/home/dmitryneuro/colab/env/bin/python3.7 -m pip install --upgrade pip' command.
You should consider upgrading via the '/home/dmitryneuro/colab/env/bin/python3.7 -m pip install --upgrade pip' command.


In [17]:
#import imageio
#fps=10
#out = imageio.get_writer('/content/out/output.mp4', mode='I', fps=fps, codec='libx264', bitrate='16M')

#for img in imgs:
#  out.append_data(np.asarray(img))
#out.close()
 

In [18]:
bands = [[8.,12.]]
#bands = [[4.,7.],[8.,12.],[13.,29.]]
#bands = [[8.,12.],[8.,12.],[8.,12.]]
#bands_name = ['theta','alpha','beta']
datas=[]
for band in range(len(bands)):
# datas.append(raw)
 datas.append(raw.pick(ch_names))
 
# datas.append(raw.filter(l_freq=bands[band][0], h_freq=bands[band][1],method='iir'))
#theta_data = raw.filter(l_freq=4, h_freq=7,method='iir')
#alpha_data = raw.filter(l_freq=8, h_freq=12,method='iir')
#beta_data = raw.filter(l_freq=13, h_freq=29,method='iir')

In [19]:
methods = ['coh']
#methods = ['wpli']
#methods = ['coh', 'plv', 'ciplv', 'ppc', 'pli', 'wpli']
epochs = []
for band in range(len(bands)):
# epochs.append(mne.make_fixed_length_epochs(datas[band], 
#                                            duration=0.1, preload=False))
 epochs.append(mne.make_fixed_length_epochs(datas[band], 
                                            duration=5*1/8, preload=False, overlap=5*1/8-0.1))
# epochs.append(mne.make_fixed_length_epochs(datas[band], duration=1.25, preload=False, overlap=1.15))
#epochs = [mne.make_fixed_length_epochs(theta_data, duration=0.1, preload=False),
#          mne.make_fixed_length_epochs(alpha_data, duration=0.1, preload=False),
#          mne.make_fixed_length_epochs(beta_data, duration=0.1, preload=False)]
#epochs = mne.make_fixed_length_epochs(alpha_data, duration=0.1, preload=False)
#epochs = mne.make_fixed_length_epochs(raw, duration=2, preload=True)
#epochs = mne.make_fixed_length_epochs(raw, duration=10, preload=True)
#epochs = mne.make_fixed_length_epochs(raw, duration=1, preload=False)
#event_related_plot = epochs.plot_image(picks=['FP1'])

Not setting metadata
Not setting metadata
17224 matching events found
No baseline correction applied
0 projection items activated


In [20]:
len(epochs[0].events)

17224

In [21]:
#fmin=8.
#fmax=13.
fmin=bands[0][0]
fmax=bands[0][1]+1.
          
sfreq = raw.info['sfreq']  # the sampling frequency
#con_methods = ['coh', 'pli', 'ciplv']
con_methods = ['coh', 'plv', 'ciplv', 'ppc', 'pli', 'wpli']
#con_methods = ['coh', 'plv', 'ciplv', 'ppc', 'pli', 'pli2_unbiased', 'wpli', 'wpli2_debiased']
#con, freqs, times, n_epochs, n_tapers = spectral_connectivity(
#    epochs, method=con_methods, mode='multitaper', sfreq=sfreq, fmin=fmin,
#    fmax=fmax, faverage=True, mt_adaptive=True, n_jobs=10)
#con, freqs, times, n_epochs, n_tapers = spectral_connectivity(
#    epochs[1][2000:2010], method=con_methods, mode='multitaper', sfreq=sfreq, 
#    faverage=True, mt_adaptive=True, n_jobs=1)
#con, freqs, times, n_epochs, n_tapers = spectral_connectivity(
#    epochs[0][2000,2001], method=con_methods, mode='multitaper', sfreq=sfreq, fmin=fmin,
#    fmax=fmax, faverage=True, mt_adaptive=True, n_jobs=1)
con, freqs, times, n_epochs, n_tapers = spectral_connectivity(
    epochs[0][2000:2010], method=con_methods, mode='multitaper', sfreq=sfreq, fmin=fmin,
    fmax=fmax, faverage=True, mt_adaptive=True, n_jobs=1)

Connectivity computation...
only using indices for lower-triangular matrix
    computing connectivity for 153 connections
    using t=0.000s..0.623s for estimation (320 points)
    frequencies: 8.0Hz..12.8Hz (4 points)
    connectivity scores will be averaged for each band
    Using multitaper spectrum estimation with 7 DPSS windows
    the following metrics will be computed: Coherence, PLV, ciPLV, PPC, PLI, WPLI
    computing connectivity for epoch 1
    computing connectivity for epoch 2
    computing connectivity for epoch 3
    computing connectivity for epoch 4
    computing connectivity for epoch 5
    computing connectivity for epoch 6
    computing connectivity for epoch 7
    computing connectivity for epoch 8
    computing connectivity for epoch 9
    computing connectivity for epoch 10
    assembling connectivity matrix
[Connectivity computation done]


In [22]:
con_res = dict()
for method, c in zip(con_methods, con):
    con_res[method] = c[:, :, 0]

In [23]:
sfreq

512.0

In [24]:
con_res


{'coh': array([[0.        , 0.        , 0.        , 0.        , 0.        ,
         0.        , 0.        , 0.        , 0.        , 0.        ,
         0.        , 0.        , 0.        , 0.        , 0.        ,
         0.        , 0.        , 0.        ],
        [0.84133218, 0.        , 0.        , 0.        , 0.        ,
         0.        , 0.        , 0.        , 0.        , 0.        ,
         0.        , 0.        , 0.        , 0.        , 0.        ,
         0.        , 0.        , 0.        ],
        [0.68781891, 0.74782064, 0.        , 0.        , 0.        ,
         0.        , 0.        , 0.        , 0.        , 0.        ,
         0.        , 0.        , 0.        , 0.        , 0.        ,
         0.        , 0.        , 0.        ],
        [0.85441927, 0.89381137, 0.78534189, 0.        , 0.        ,
         0.        , 0.        , 0.        , 0.        , 0.        ,
         0.        , 0.        , 0.        , 0.        , 0.        ,
         0.        , 0.    

In [25]:
from mne.viz import circular_layout, plot_connectivity_circle
#import matplotlib.pyplot as plt

label_names = ch_names

#plot_connectivity_circle(con_res['pli'], label_names, 
#                                                  title='All-to-All Connectivity (PLI)')
#fig = plt.figure(num=None, figsize=(8, 4), facecolor='black')
no_names = [''] * len(label_names)
for ii, method in enumerate(con_methods):
    plot_connectivity_circle(con_res[method], label_names, n_lines=300,
                             title=method)
    #plot_connectivity_circle(con_res[method], label_names, n_lines=300,
    #                         title=method, padding=0, fontsize_colorbar=6,
    #                         fig=fig, subplot=(1, 8, ii + 1))

    #plot_connectivity_circle(con_res[method], no_names, n_lines=300,
    #                         title=method, padding=0, fontsize_colorbar=6,
    #                         fig=fig, subplot=(1, 6, ii + 1))
plt.show()

/home/dmitryneuro/colab/env/lib/python3.7/site-packages/ipykernel_launcher.py:20: UserWarning: Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.


In [26]:
con_res['coh']

array([[0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        ],
       [0.84133218, 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        ],
       [0.68781891, 0.74782064, 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        ],
       [0.85441927, 0.89381137, 0.78534189, 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        ],
    

In [27]:
import io
from PIL import Image

#%matplotlib inline
#import matplotlib.pyplot as plt

In [28]:
#import matplotlib
#matplotlib.use('agg')


In [29]:
#matplotlib.interactive(False)

In [30]:
#import gc

import imageio

!pip install mne-connectivity

from mne_connectivity.viz import plot_sensors_connectivity


You should consider upgrading via the '/home/dmitryneuro/colab/env/bin/python3.7 -m pip install --upgrade pip' command.


In [ ]:
#matplotlib.use('agg')
#fig = plt.figure()

fps=10
#out = imageio.get_writer('/content/out/output.mp4', fps=fps)
#out = imageio.get_writer('/content/out/'+input_fname_name+'_circle_'+methods[0]+'_'+
#                         str(int(bands[0][0]))+'-'+str(int(bands[0][1]))+'hz_'+str(len(epochs[0].events)-2)+'.mp4', fps=fps)
#out = imageio.get_writer('/content/out/output.mp4', mode='I', fps=fps, codec='libx264', bitrate='16M')

# Generate random breaks and display audio

# CHANGE THIS to change number of examples generated
#n_generate = 30
#n_generate = 150
#n_generate = 300
#n_generate = 305
#n_generate = 390

# Sample latent vectors
#seed = 666 # change this seed to generate different set of breaks
#np.random.seed(seed)
#_z = (np.random.rand(n_generate, dim) * 2.) - 1.


hz=44100
#hz=39936
#hz=int(32768*2*(600/240))
#hz=int(32768*2*(480/240))
#hz=int(32768*2*(360/240))
#hz=int(32768*2*(300/240))
#hz=int(32768*2*(265/240))
#hz=int(32768*2*(250/240))
#hz=int(32768*2*(240/240)*1.6666666)
#hz=int(32768*2*(240/240))
#hz=int(32768*2*(120/240))
#fps=hz/(32768*2)
#fps=10
#fps=0.5
#fps=44100/(32768*2)
#fps=1
#fps=1/3
#fps=1.5

#n_generate=int((307-2)*fps)
#n_generate=int((160-2)*fps)
#n_generate=int((1598-2)*fps)
#n_generate=int((1648-2)*fps)
n_generate=len(epochs[0].events)-2

#n_generate=int((10-2)*fps)
#n_generate=int((1607-2)*fps)
#n_generate=150
part_len = 1000
#part_len = 275
dim = 512


n_parts = n_generate//part_len
if n_generate%part_len>0:
    n_parts=n_parts+1

vol=0.1

#psd_array=np.random.rand(part_len, dim) 

#out = imageio.get_writer('./output.mp4', mode='I', fps=fps, codec='libx264', bitrate='16M')

#Gs_kwargs = dnnlib.EasyDict()
#Gs_kwargs.output_transform = dict(func=tflib.convert_images_to_uint8, nchw_to_nhwc=True)
#Gs_kwargs.randomize_noise = False

imgs = []
imgs1 = []

def plot_func(con,methods,label_names,out):
        #plt.ioff()
        if True:
#          con_res = dict()
#          for method, c in zip(methods, con):
#            con_res[method] = c[:, :, 0]
#            con_res[method] = c[:, :]
          #for ii, method in enumerate(methods):
#          if False:
          if True:
#            fig,_ = plot_connectivity_circle(con_res[method], label_names, n_lines=300, 

#            plot_connectivity_circle(con[:, :, 0], label_names, n_lines=300, 
#                                             title=method, show = False)
            #fig = plt.figure()
            #fig,_ = 
#            fig,ax = plot_connectivity_circle(con[:, :, 0], label_names, n_lines=300, 
#                                             title=method, show = False, vmin=0, vmax=1)#, fig=fig)
            #fig, ax = plt.subplots(figsize=(37.33, 21)) 
#            fig,ax = plot_connectivity_circle(con[:, :, 0], label_names, n_lines=300, 
#                                             title=methods[0], show = False, vmin=0, vmax=1, fontsize_names=16)#, fig=fig)
            fig,ax = plot_connectivity_circle(con[:, :, 0], label_names, n_lines=300, 
                                             title=input_fname_name+'_circle_'+methods[0]+'_'+str(int(bands[0][0]))+'-'+str(int(bands[0][1]))+'hz_'+str(len(epochs[0].events)-2), 
                                             show = False, vmin=0, vmax=1, fontsize_names=16)#, fig=fig)
            #ax.set_theta_zero_location("W")
            
            #ax.get_th
            #ax.set_rotation()
            #print(ax.get_xticks())
#            ax.set_theta_zero_location((90+360/len(label_names)))
            #print(ax.get_xticklabels())
            #print(ax.get_xticks())
            #xticks=ax.get_xticks()
            #xticks.
            #ax.set_rotation(0)

            #rtick_locs = range(len(label_names))
            #ax.set_rgrids(rtick_locs)
            #ax.set_rticks(rtick_locs)
            
            #set_rgrids(self, radii, labels=None, angle=None, fmt=None, **kwargs)

            #thetatick_locs = range(len(label_names))
#            thetatick_locs = np.linspace(0.,45.,4)
            #thetatick_labels = [u'%i\u00b0'%np.round(x) for x in thetatick_locs]
            #ax.set_thetagrids(thetatick_locs, thetatick_labels, fontsize=16)

            #set_thetagrids(self, angles, labels=None, fmt=None, **kwargs)[source]
            
            #ax.set_xticks(np.linspace(0, 2*np.pi, len(label_names), endpoint=False))
            #ax.set_xticklabels(range(len(label_names)))  

#            ax.set_theta_offset(np.deg2rad(90+360/len(label_names)))
            ax.set_theta_offset(np.deg2rad(360/len(label_names)))
#            ax.set_thetagrids(range(len(label_names)), 
#                  labels=range(len(label_names)), fontsize=12)

            #ax.set_rgrids(range(len(label_names)), labels=range(len(label_names)), fontsize=12, angle=180)

            
            #ax.set_xticklabels(ax.get_xticks(), rotation = np.deg2rad(90+360/len(label_names)))
            #for tick in ax.get_xticklabels():
            #  tick.set_rotation(45)
#            for label, angle in zip(ax.get_xticklabels(), angles):
            #labels = []
            #for label in zip(ax.get_xticklabels()):
            #  x,y = label.get_position()
            #  lab = ax.text(x,y, label.get_text(), transform=label.get_transform(),
            #                ha=label.get_ha(), va=label.get_va())
            #  #lab.set_rotation(angle)
            #  lab.set_rotation(10)
            #  labels.append(lab)
            ##ax.set_xticklabels([])
            #ax.set_xticklabels(labels)
            #plt.show()  

#            fig = plot_sensors_connectivity(raw.info, con[:, :, 0], picks=label_names, cbar_label=method)
          #plot_conmat_file = os.path.abspath('circle_' + fname + '.eps')
          #fig.savefig(plot_conmat_file, facecolor='black')

          #plt.close()
          #plt.close(fig)
          #fig.clf()
          
          #del fig

          #if ji%100==0 :
          #  gc.collect()

        #if False:

          #plt.close(fig)
          ##fig1.close()
          #del fig

          #plt.rcParams["figure.figsize"] = [7.50, 3.50]
          #plt.rcParams["figure.autolayout"] = True

          #plt.figure()
          #plt.plot([1, 2])

          img_buf = io.BytesIO()
          #plt.savefig(img_buf, format='png')

          fig.savefig(img_buf, facecolor='black', format='png')
          fig.savefig('/content/out/img_buf.png', facecolor='black', format='png')

          #fig.clear()
          #fig.clf()
          #plt.close()
          #ax.cla()

          #plt.savefig(img_buf, facecolor='black', format='png')
          #plt.savefig('/content/out/img_buf.png', facecolor='black', format='png')

          #plt.close()
          #plt.close(fig)
          #fig.clf()

          # Clear the current axes.
          #plt.cla() 
          # Clear the current figure.
          #plt.clf() 
          # Closes all the figure windows.
          #plt.close('all')   
          #plt.close(fig)
          #gc.collect()
          #del fig

          #if ji%100==0 :
          #  gc.collect()

          img_buf.seek(0)

          im1 = Image.open(img_buf)
          im1.crop(0,0,576,576)
          img_buf1 = io.BytesIO()
          im1.write(img_buf1)
          img_buf1.seek(0)

          im = imageio.imread(img_buf1)
          out.append_data(im)
          img_buf.close()
          img_buf1.close()

          plt.close('all')   

#z_avg_samples=n_generate
#for i in range(n_generate): # display separate audio for each break
for j in range(n_parts): # display separate audio for each break
  out_file = '/content/out/'+input_fname_name+'_circle_'+methods[0]+'_'+str(int(bands[0][0]))+'-'+str(int(bands[0][1]))+'hz_'+str(len(epochs[0].events)-2)+'_'+str(j)+'.mp4'
  out_file_parts = '/content/out/parts/'+input_fname_name+'_circle_'+methods[0]+'_'+str(int(bands[0][0]))+'-'+str(int(bands[0][1]))+'hz_'+str(len(epochs[0].events)-2)+'_'+str(j)+'.mp4'

  import os.path
  if not os.path.isfile(out_file_parts):  
    if os.path.isfile(out_file):  
      !rm {out_file}
    out = imageio.get_writer(out_file, fps=fps)

    for i in range(part_len): # display separate audio for each break
        ji = j * part_len + i
        
#        if (i==0) and (n_generate-ji<part_len):
#            psd_array=np.random.rand((n_generate-ji), dim) 


        eeg_step=ji
        #print (f'EEG step: {(eeg_step/3):.1f} s')
        tmin, tmax = 0+(eeg_step/fps), 2+(eeg_step/fps)  # use the first 120s of data
        #tmin, tmax = 0+(10*eeg_step/512), 2+(10*eeg_step/512)  # use the first 120s of data
        #fmin, fmax = 0.5, 256  # look at frequencies between 4 and 100Hz
        #fmin, fmax = 0.5, 50  # look at frequencies between 4 and 100Hz
        #fmin, fmax = 8, 12  # look at frequencies between 4 and 100Hz
        #n_fft = 512  # the FFT size (n_fft). Ideally a power of 2
        #n_fft = 1024  # the FFT size (n_fft). Ideally a power of 2
        #n_fft = 2048  # the FFT size (n_fft). Ideally a power of 2
        #label = mne.read_label(fname_label)
        
        print(str(ji) + '/' + str(n_generate))
        #logger = logging.getLogger()
        #logger.disabled = True

        sfreq = raw.info['sfreq']  # the sampling frequency
        
#        psds=np.zeros(dim)
        
        for method in range(len(methods)):
         for band in range(len(bands)):
          #fmin=8.
          #fmax=13.
          fmin=bands[band][0]
          fmax=bands[band][1]
          if band == 0:
           con, freqs, times, n_epochs, n_tapers = spectral_connectivity(
            epochs[band][ji:ji+1], method=methods[method], mode='multitaper', sfreq=sfreq, fmin=fmin,
#            epochs[band][ji:ji+10], method=methods[method], mode='multitaper', sfreq=sfreq, fmin=fmin,
            fmax=fmax, faverage=True, mt_adaptive=True, n_jobs=1, verbose=50)
          #con, freqs, times, n_epochs, n_tapers = spectral_connectivity(
          #  epochs[band][ji,ji+1], method=methods[method], mode='multitaper', sfreq=sfreq, fmin=fmin,
          #  fmax=fmax, faverage=True, mt_adaptive=True, n_jobs=1)
#          psds_shift1=int(round(method*len(bands)+band)*(len(ch_names)*(len(ch_names)-1)/2))
#          ji1=0
#          for j1 in range(0,len(ch_names)): # display separate audio for each break
#            for i1 in range(0,j1): # display separate audio for each break
#              psds[ji1+psds_shift1]=(con[j1][i1][0]-0.5)*1
#              ji1 = ji1+1
        #psds, freqs = mne.time_frequency.psd_welch(raw, picks=picks,
        #                 tmin=tmin, tmax=tmax, fmin=fmin, fmax=fmax,
        #                 n_fft=n_fft)
        #logger.disabled = False

        #print(freqs)
        #print(psds)
        
#        psd_array[i]=psds
        ##z_samples = psds

        #w_samples = G.mapping(torch.from_numpy(z_samples).to(device), None)  # [N, L, C]
        #w_samples = w_samples[:, :1, :].cpu().numpy().astype(np.float32)       # [N, 1, C]
        ##z_avg = np.mean(z_samples, axis=0)      # [1, 1, C]
        #z_avg = np.mean(z_samples, axis=0, keepdims=True)      # [1, 1, C]
        ##psd_array[i]=z_avg
        #psd_array[i]=z_avg
        #print(z_avg)
        #z_std = (np.sum((z_samples - z_avg) ** 2) / z_avg_samples) ** 0.5

        #psd_array[i]=psds
        #psds_transpose=np.transpose(psds)
        #plt.plot(freqs,psds_transpose)
        #plt.xlabel('Frequency (Hz)')
        #plt.ylabel('PSD (dB)')
        #plt.title('Power Spectrum (PSD)')
        #plt.show()
#        if (i==part_len-1) or (ji==n_generate-1) :
        if True:
#        if False:
#          con_res = dict()
#          for method, c in zip(methods, con):
#            con_res[method] = c[:, :, 0]
#            con_res[method] = c[:, :]
          for ii, method in enumerate(methods):
#            fig,_ = plot_connectivity_circle(con_res[method], label_names, n_lines=300, 

#            plot_connectivity_circle(con[:, :, 0], label_names, n_lines=300, 
#                                             title=method, show = False)
            #px = 1/plt.rcParams['figure.dpi']  # pixel in inches
            #fig = plt.figure(figsize=(576*px, 576*px))
#            fig,_ = 

#            fig,ax = plot_connectivity_circle(con[:, :, 0], label_names, n_lines=300, 
#                                             title=method, show = False, vmin=0, vmax=1)#, fig=fig)
            fig,ax = plot_connectivity_circle(con[:, :, 0], label_names, n_lines=300, 
#                                             title=input_fname_name+'_circle_'+methods[0]+'_'+str(int(bands[0][0]))+'-'+str(int(bands[0][1]))+'hz_'+str(len(epochs[0].events)-2), 
                                             title=input_fname_name+'_circle_'+methods[0]+'_'+str(int(bands[0][0]))+'-'+str(int(bands[0][1]))+'hz_'+str(len(epochs[0].events)-2)+'\n'+str(ji), 
#                                             title=input_fname_name+'_circle_'+methods[0]+'_'+str(int(bands[0][0]))+'-'+str(int(bands[0][1]))+'hz_'+str(len(epochs[0].events)-2)+'_'+str(ji), 
                                             show = False, vmin=0, vmax=1, fontsize_names=16)#, fig=fig)
            #ax.set_theta_offset(np.deg2rad(360/len(label_names)))
#            fig = plot_sensors_connectivity(raw.info, con[:, :, 0], picks=label_names, cbar_label=method)
          #plot_conmat_file = os.path.abspath('circle_' + fname + '.eps')
          #fig.savefig(plot_conmat_file, facecolor='black')

          #plt.close()
          #plt.close(fig)
          #fig.clf()
          
          #del fig

          #if ji%100==0 :
          #  gc.collect()

#        if False:
        if True:

          #plt.close(fig)
          ##fig1.close()
          #del fig

          #plt.rcParams["figure.figsize"] = [7.50, 3.50]
          #plt.rcParams["figure.autolayout"] = True

          #plt.figure()
          #plt.plot([1, 2])

          img_buf = io.BytesIO()
          #plt.savefig(img_buf, format='png')

          #fig.savefig(img_buf, facecolor='black', format='png')
          #fig.savefig('/content/out/img_buf.png', facecolor='black', format='png')

          #fig.clf()
          #plt.close()
          #ax.cla()

          #import matplotlib.transforms
          #plt.savefig(img_buf, facecolor='black', format='png', 
          #            bbox_inches=matplotlib.transforms.Bbox([[100, 100], [100, 100]]))
          #fig.set_size_inches(8.2, 8.11)

#          plt.savefig(img_buf, facecolor='black', format='png', bbox_inches='tight')
          plt.savefig(img_buf, facecolor='black', format='png')#, bbox_inches='tight')
          #plt.savefig('/content/out/img_buf.png', facecolor='black', format='png')

          #plt.close()
          #plt.close(fig)
          #fig.clf()

          # Clear the current axes.
          #plt.cla() 
          # Clear the current figure.
          #plt.clf() 
          # Closes all the figure windows.
          #plt.close('all')   
          #plt.close(fig)
          #gc.collect()
          #del fig
          #del ax

          #if ji%100==0 :
          #  gc.collect()

          img_buf.seek(0)

          im1 = Image.open(img_buf)

          size = 432
          im2 = im1.crop((35, 70, 35+size, 70+size))
          img_buf1 = io.BytesIO()
          im2.save(img_buf1, format='png')
          img_buf1.seek(0)

          im = imageio.imread(img_buf1)
          #out.append_data(im)
          img_buf.close()
          img_buf1.close()

#          im = imageio.imread(img_buf)
#          img_buf.close()

          #print(im.shape)
          #(412, 399, 4)
          im_arr = np.array(im)
          #print(im_arr.shape)
          #im_arr.reshape(416, 416, 4)
          im_arr_rot = np.rot90(im_arr)
          #im_arr_rot.resize((400, 416))

          out.append_data(im_arr_rot)
          imageio.imwrite('/content/out/img_buf.png', im_arr_rot, format='png')
#          out.append_data(im)
          #del img_buf
          #del im

          #if ji == 100:
          #  out.close()
          #  break

          #im = img_buf.getvalue()

          #im = Image.open(img_buf)
#          im.copy()
          #im.show(title="My Image")
          
          #out.append_data(np.asarray(im))

          #im.close()

          #img_buf.close()

          #  _z = psd_array * vol
          #  images = Gs.run(_z, None, **Gs_kwargs) # [minibatch, height, width, channel]
          #  for image in images:
              #imgs1.append(image)
#              out.append_data(np.asarray(PIL.Image.fromarray(image, 'RGB')))

              #imgs.append(PIL.Image.fromarray(image, 'RGB'))
            #out.append(PIL.Image.fromarray(images[0], 'RGB'))
            #_G_z = sess.run(G_z, {z: _z})[:,:,0]
            #if j==0:
            #    _G_z_full=_G_z
            #else:
            #    _G_z_full=np.append(_G_z_full,_G_z)

        #if True:
        #  plot_func(con,methods,label_names,out)
          #if ji%100==0 :
          #  gc.collect()
          if (ji==n_generate-1) :
                break
    out.close()
    !mv {out_file} {out_file_parts}


#print(psd_array)
#print(psd_array.shape)
#print(psd_array.ndim)
#_z = psd_array / 5.
#_z = psd_array / 10.
#_z = (psd_array * 2.) - 1.
#_G_z = sess.run(G_z, {z: _z})[:,:,0]

# display(Audio(_G_z.flatten(), rate=39936)) # display all in one audio

#for i in range(n_generate): # display separate audio for each break
  #print(i)
  #display(Audio(np.tile(_G_z[i][1:_G_z[i].ndim/2], 2), rate=39936)) # change rate for different tempo
  #display(Audio(np.tile(_G_z[i][1:32768], 2), rate=32768)) # change rate for different tempo
  #display(Audio(np.tile(_G_z[i], 1), rate=32768)) # change rate for different tempo
#out.close()

0/17222
1/17222
2/17222
3/17222
4/17222
5/17222
6/17222
7/17222
8/17222
9/17222
10/17222
11/17222
12/17222
13/17222
14/17222


/home/dmitryneuro/colab/env/lib/python3.7/site-packages/mne/viz/circle.py:278: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  fig = plt.figure(figsize=(8, 8), facecolor=facecolor)


15/17222
16/17222
17/17222
18/17222
19/17222
20/17222
21/17222
22/17222
23/17222
24/17222
25/17222
26/17222
27/17222
28/17222
29/17222
30/17222
31/17222
32/17222
33/17222
34/17222
35/17222
36/17222
37/17222
38/17222
39/17222
40/17222
41/17222
42/17222
43/17222
44/17222
45/17222
46/17222
47/17222
48/17222
49/17222
50/17222
51/17222
52/17222
53/17222
54/17222
55/17222
56/17222
57/17222
58/17222
59/17222
60/17222
61/17222
62/17222
63/17222
64/17222
65/17222
66/17222
67/17222
68/17222
69/17222
70/17222
71/17222
72/17222
73/17222
74/17222
75/17222
76/17222
77/17222
78/17222
79/17222
80/17222
81/17222
82/17222
83/17222
84/17222
85/17222
86/17222
87/17222
88/17222
89/17222
90/17222
91/17222
92/17222
93/17222
94/17222
95/17222
96/17222
97/17222
98/17222
99/17222
100/17222
101/17222
102/17222
103/17222
104/17222
105/17222
106/17222
107/17222
108/17222
109/17222
110/17222
111/17222
112/17222
113/17222
114/17222
115/17222
116/17222
117/17222
118/17222
119/17222
120/17222
121/17222
122/17222
123/1

In [ ]:
!mkdir '/content/out/parts'
out_file_parts_txt = '/content/out/parts/'+input_fname_name+'_circle_'+methods[0]+'_'+str(int(bands[0][0]))+'-'+str(int(bands[0][1]))+'hz_'+str(len(epochs[0].events)-2)+'.txt'
!rm {out_file_parts_txt}
out_file_mp4 = '/content/out/'+input_fname_name+'_circle_'+methods[0]+'_'+str(int(bands[0][0]))+'-'+str(int(bands[0][1]))+'hz_'+str(len(epochs[0].events)-2)+'.mp4'
with open(out_file_parts_txt, 'a') as the_file:
 for j in range(n_parts): # display separate audio for each break
  out_file_part = '/content/out/parts/'+input_fname_name+'_circle_'+methods[0]+'_'+str(int(bands[0][0]))+'-'+str(int(bands[0][1]))+'hz_'+str(len(epochs[0].events)-2)+'_'+str(j)+'.mp4'
  the_file.write("file '")
  the_file.write(out_file_part)
  the_file.write("'\n")
#  import os.path
#  if not os.path.isfile(out_file):  
#    out = imageio.get_writer(out_file, fps=fps)

!ffmpeg -f concat -safe 0 -i {out_file_parts_txt} -c copy {out_file_mp4}
#!ffmpeg -vfilters "rotate=90" -i {out_file_mp4} {out_file_mp4}


In [ ]:
stop

In [ ]:
for size in range(20):
          img_buf = io.BytesIO()

#          fig.set_size_inches(5.0+size/10.0,5.0+size/10.0)
#          fig.set_size_inches(5.0+15/10.0,5.0+15/10.0)
          #fig.set_size_inches(6.1,6.1)
#          fig.set_size_inches(6.5,6.5)
          fig.set_dpi(70+size)
          print(size)
#          fig.set_size_inches(65,65)

          plt.savefig(img_buf, facecolor='black', format='png')#, bbox_inches='tight')

          img_buf.seek(0)

          im = imageio.imread(img_buf)
          img_buf.close()

          print(im.shape)
          print(im.shape[0]/16)
          print(im.shape[1]/16)
          

In [ ]:
out.close()

In [ ]:
print(plt.rcParams["savefig.bbox"])

In [ ]:
input_fname=files_path[3][1]
input_fname_name=files_path[3][2]

In [ ]:
%cd '/content/out/'

input0 = '20211130-162023-sync1_circle_coh_8-12hz_23692.mp4'
input1 = '20211130-162023-sync2_circle_coh_8-12hz_23692.mp4'
input2 = '20211130-162023-sync3_circle_coh_8-12hz_23692.mp4'
input3 = '20211130-162023-sync4_circle_coh_8-12hz_23692.mp4'
output = '20211130-162023-sync_circle_coh_8-12hz_23692.mp4'

!docker run -v $(pwd):$(pwd) -w $(pwd) jrottenberg/ffmpeg -i {input0} -i {input1} -i {input2} -i {input3} -filter_complex "[0:v][1:v][2:v][3:v]xstack=inputs=4:layout=0_0|w0_0|0_h0|w0_h0[v]" -map "[v]" {output}

In [ ]:
input0 = '/content/out/parts/'+'20211130-162023-sync1_circle_coh_8-12hz_23692_0.mp4'
input1 = '/content/out/parts/'+'20211130-162023-sync2_circle_coh_8-12hz_23692_0.mp4'
input2 = '/content/out/parts/'+'20211130-162023-sync3_circle_coh_8-12hz_23692_0.mp4'
input3 = '/content/out/parts/'+'20211130-162023-sync4_circle_coh_8-12hz_23692_0.mp4'
output = '/content/out/parts/'+'20211130-162023-sync_circle_coh_8-12hz_23692_0.mp4'

!ffmpeg -i {input0} -i {input1} -i {input2} -i {input3} -filter_complex "[0:v][1:v][2:v][3:v]xstack=inputs=4:layout=0_0|w0_0|0_h0|w0_h0[v]" -map "[v]" {output}

In [ ]:
#!pip install pyvista

In [ ]:
#import pyvista

In [ ]:
!ffmpeg -version